In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd /content/drive/MyDrive/NLP_Project/

/content/drive/.shortcut-targets-by-id/1liGtjC9dkd3fXcYr3IVfYFk0iZ6z7rjm/NLP_Project


In [5]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv("mpst_full_data.csv")
df

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb
...,...,...,...,...,...,...
14823,tt0219952,Lucky Numbers,"In 1988 Russ Richards (John Travolta), the wea...","comedy, murder",test,wikipedia
14824,tt1371159,Iron Man 2,"In Russia, the media covers Tony Stark's discl...","good versus evil, violence",train,wikipedia
14825,tt0063443,Play Dirty,During the North African Campaign in World War...,anti war,train,wikipedia
14826,tt0039464,High Wall,Steven Kenet catches his unfaithful wife in th...,murder,test,wikipedia


In [7]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [8]:
from nltk.corpus import stopwords 

In [9]:
!pip install textsearch
!pip install contractions
!pip install tqdm
import nltk
nltk.download('punkt')

     |████████████████████████████████| 286kB 10.8MB/s 
     |████████████████████████████████| 327kB 18.8MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85392 sha256=797418c471e8a06cc82066c011d909985f54ea240d8e8e6a0a55bca20a3e2552
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
stopwords=set(stopwords.words('english'))

In [11]:
import re
import contractions
from bs4 import BeautifulSoup
preprocessed=[]
for i in df['plot_synopsis'].values:
    i= re.sub(r"http\S+","", i)
    i= BeautifulSoup(i, 'lxml').get_text()
    i= re.sub("\S*\d\S*", "",i).strip()
    i= re.sub('[^A-Za-z]+', ' ', i)
    i= contractions.fix(i)
    i= ' '.join(j.lower() for j in i.split() if j.lower() not in stopwords)
    preprocessed.append(i.strip())
df['preprocessed_plots']=preprocessed

In [12]:
check=["one","two","three","four","five","six","seven","eight","nine","ten","iii","i","ii","iv","v","vi","vii","viii","ix","x"]
preprocessed_title=[]
for i in df['title'].values:
    i= re.sub(r"http\S+","", i)
    i= BeautifulSoup(i, 'lxml').get_text()
    i= re.sub("\S*\d\S*", "",i).strip()
    i= re.sub('[^A-Za-z]+', ' ', i)
    i= contractions.fix(i)
    i= ' '.join(j.lower() for j in i.split() if j.lower() not in stopwords)
    g=i.split()
    if("part" in g):
      ind=g.index("part")
      if((ind+1<len(g)-1) and (g[ind+1] in check)):
        g.remove(g[ind])
        g.remove(g[ind+1])
    i= ' '.join(g)
    preprocessed_title.append(i.strip())
df['preprocessed_title']=preprocessed_title

In [13]:
def remove_spaces(x):
    return x.replace(" ", "")
df['tags']=df['tags'].apply(remove_spaces)
df['tags']

0                    cult,horror,gothic,murder,atmospheric
1                                                 violence
2                                                 romantic
3                      inspiring,romantic,stupid,feel-good
4        cruelty,murder,dramatic,cult,violence,atmosphe...
                               ...                        
14823                                        comedy,murder
14824                              goodversusevil,violence
14825                                              antiwar
14826                                               murder
14827                                        christianfilm
Name: tags, Length: 14828, dtype: object

In [14]:
df.head()

,imdb_id,title,plot_synopsis,tags,split,synopsis_source,preprocessed_plots,preprocessed_title
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult,horror,gothic,murder,atmospheric",train,imdb,note synopsis orginal italian release segments...,tre volti della paura
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb,two thousand years ago nhagruul foul sorcerer ...,dungeons dragons book vile darkness
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb,matuschek gift store budapest workplace alfred...,shop around corner
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring,romantic,stupid,feel-good",train,imdb,glenn holland morning person anyone standards ...,mr holland opus
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty,murder,dramatic,cult,violence,atmosphe...",val,imdb,may cuban man named tony montana al pacino cla...,scarface


In [15]:
train, validate, test = np.split(df.sample(frac=1), [int(.7*len(df)), int(.85*len(df))])

In [16]:
print(train.shape)
print(test.shape)
print(validate.shape)

(10379, 8)
(2225, 8)
(2224, 8)


In [17]:
train=train.reset_index()
test=test.reset_index()
validate=validate.reset_index()

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(","), binary='true')
y_train = vectorizer.fit_transform(train['tags']).toarray()
y_test = vectorizer.transform(test['tags']).toarray()
y_val = vectorizer.transform(validate['tags']).toarray()

In [ ]:
x=train['plot_synopsis']

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
vect=Tokenizer()
vect.fit_on_texts(train['preprocessed_plots'])
vocab_size = len(vect.word_index) + 1
vocab_size

102840

In [ ]:
encoded_docs_train = vect.texts_to_sequences(train['preprocessed_plots'])
max_length = vocab_size
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=1200)
print(padded_docs_train)

[[    0     0     0 ...   100 28474   843]
 [    0     0     0 ...  2503   512 57521]
 [    0     0     0 ...  4314    66  1318]
 ...
 [    0     0     0 ...   943     3   336]
 [    0     0     0 ...   680    69    16]
 [    0     0     0 ...  4995  1060  4812]]


In [ ]:
encoded_docs_test =  vect.texts_to_sequences(test['preprocessed_plots'])
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=1200)
encoded_docs_cv = vect.texts_to_sequences(validate['preprocessed_plots'])
padded_docs_cv = pad_sequences(encoded_docs_cv, maxlen=1200)

In [ ]:
model = keras.Sequential()
model.add(layers.Embedding(vocab_size, output_dim=50, input_length=1200))
model.add(layers.LSTM(128, return_sequences=True))  
model.add(layers.Dropout(0.5))
model.add(layers.LSTM(64))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(71, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1200, 50)          5142000   
_________________________________________________________________
lstm (LSTM)                  (None, 1200, 128)         91648     
_________________________________________________________________
dropout (Dropout)            (None, 1200, 128)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 71)                4615      
Total params: 5,287,671
Trainable params: 5,287,671
Non-trainable params: 0
______________________________________________

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['binary_accuracy'])

In [ ]:
history = model.fit(padded_docs_train,y_train,
                        epochs = 5,
                        verbose = 1,
                        validation_data=(padded_docs_cv, y_val),
                        batch_size=16)

Epoch 1/5
649/649 [==============================] - 114s 124ms/step - loss: 0.2375 - binary_accuracy: 0.9183 - val_loss: 0.1396 - val_binary_accuracy: 0.9578
Epoch 2/5
649/649 [==============================] - 79s 122ms/step - loss: 0.1467 - binary_accuracy: 0.9576 - val_loss: 0.1395 - val_binary_accuracy: 0.9578
Epoch 3/5
649/649 [==============================] - 79s 122ms/step - loss: 0.1777 - binary_accuracy: 0.9524 - val_loss: 0.1395 - val_binary_accuracy: 0.9578
Epoch 4/5
649/649 [==============================] - 79s 122ms/step - loss: 0.1435 - binary_accuracy: 0.9578 - val_loss: 0.1398 - val_binary_accuracy: 0.9578
Epoch 5/5
649/649 [==============================] - 79s 122ms/step - loss: 0.1415 - binary_accuracy: 0.9578 - val_loss: 0.1404 - val_binary_accuracy: 0.9578


In [ ]:
predictions=model.predict(padded_docs_test)

In [ ]:
predictions

array([[0.00559035, 0.02060663, 0.00264743, ..., 0.26407307, 0.00133871,
        0.00135729],
       [0.00899487, 0.02995011, 0.0045598 , ..., 0.28915995, 0.00252725,
        0.00252182],
       [0.01060329, 0.03321199, 0.00553552, ..., 0.29236078, 0.00316923,
        0.00315615],
       ...,
       [0.01109217, 0.03532417, 0.00596316, ..., 0.30519316, 0.00333194,
        0.00325807],
       [0.00749082, 0.02601513, 0.00370683, ..., 0.28068453, 0.001969  ,
        0.00197729],
       [0.01589642, 0.04695798, 0.00887907, ..., 0.326617  , 0.00529375,
        0.00515014]], dtype=float32)

In [ ]:
model.evaluate(padded_docs_test,y_test)

70/70 [==============================] - 3s 39ms/step - loss: 0.1360 - binary_accuracy: 0.9594


[0.13597892224788666, 0.9594241380691528]

In [ ]:
t = train.sample(1)
encoded_docs = vect.texts_to_sequences(t['preprocessed_plots'])
padded_docs = pad_sequences(encoded_docs, maxlen=1200)
pred = model.predict(padded_docs).tolist()
for i in range(len(pred[0])):
  if(pred[0][i] < 0.1):
    pred[0][i] =0
  else:
    pred[0][i] =1
print("Original tags -->", t['tags'].values)
print("Predicted tags -->", vectorizer.inverse_transform(pred[0])[0])

Original tags --> ['alternatehistory']
Predicted tags --> ['comedy' 'cult' 'flashback' 'murder' 'psychedelic' 'revenge' 'romantic'
 'violence']


In [ ]:
pred=predictions.copy()
pred[pred>=0.1]=1
pred[pred<0.1]=0
pred=pred.astype(np.int64)
from sklearn.metrics import hamming_loss
hamming_loss(y_test,pred)

0.10651052381705967

In [ ]:
vect_1=Tokenizer()
vect_1.fit_on_texts(train['title'])
vocab_size_1 = len(vect_1.word_index) + 1

In [ ]:
encoded_docs_train_1 = vect_1.texts_to_sequences(train['preprocessed_title'])
max_length_1 = vocab_size_1
padded_docs_train_1 = pad_sequences(encoded_docs_train_1, maxlen=1200)
print(padded_docs_train_1)

[[   0    0    0 ...  179  179   45]
 [   0    0    0 ...    0  990  795]
 [   0    0    0 ...    0  377  122]
 ...
 [   0    0    0 ...  666   48 1218]
 [   0    0    0 ...    0 2501  143]
 [   0    0    0 ... 3159  511 9588]]


In [ ]:
encoded_docs_test_1 =  vect_1.texts_to_sequences(test['preprocessed_title'])
padded_docs_test_1 = pad_sequences(encoded_docs_test_1, maxlen=1200)
encoded_docs_cv_1 = vect_1.texts_to_sequences(validate['preprocessed_title'])
padded_docs_cv_1 = pad_sequences(encoded_docs_cv_1, maxlen=1200)

In [ ]:
model_1 = keras.Sequential()
# Configuring the parameters
model_1.add(layers.Embedding(vocab_size_1, output_dim=50, input_length=1200))
model_1.add(layers.LSTM(128, return_sequences=True))  
# Adding a dropout layer
model_1.add(layers.Dropout(0.5))
model_1.add(layers.LSTM(64))
model_1.add(layers.Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model_1.add(layers.Dense(71, activation='sigmoid'))
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1200, 50)          479450    
_________________________________________________________________
lstm_2 (LSTM)                (None, 1200, 128)         91648     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1200, 128)         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 71)                4615      
Total params: 625,121
Trainable params: 625,121
Non-trainable params: 0
________________________________________________

In [ ]:
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['binary_accuracy'])

In [ ]:
history_1 = model_1.fit(padded_docs_train_1,y_train,
                        epochs = 5,
                        verbose = 1,
                        validation_data=(padded_docs_test_1, y_test),
                        batch_size=16)

Epoch 1/5
649/649 [==============================] - 58s 84ms/step - loss: 0.2291 - binary_accuracy: 0.9223 - val_loss: 0.1356 - val_binary_accuracy: 0.9594
Epoch 2/5
649/649 [==============================] - 54s 83ms/step - loss: 0.1469 - binary_accuracy: 0.9572 - val_loss: 0.1355 - val_binary_accuracy: 0.9594
Epoch 3/5
649/649 [==============================] - 55s 85ms/step - loss: 0.1432 - binary_accuracy: 0.9578 - val_loss: 0.1368 - val_binary_accuracy: 0.9594
Epoch 4/5
649/649 [==============================] - 55s 85ms/step - loss: 0.1385 - binary_accuracy: 0.9577 - val_loss: 0.1390 - val_binary_accuracy: 0.9594
Epoch 5/5
649/649 [==============================] - 55s 85ms/step - loss: 0.1373 - binary_accuracy: 0.9574 - val_loss: 0.1401 - val_binary_accuracy: 0.9594


In [ ]:
predictions_1=model_1.predict(padded_docs_test_1)

In [ ]:
predictions_1

array([[3.7278634e-02, 9.2800647e-02, 2.6593812e-02, ..., 3.7665769e-01,
        1.3770727e-02, 1.2018981e-02],
       [8.2755340e-03, 3.4142859e-02, 6.4167506e-03, ..., 2.9511219e-01,
        2.7486794e-03, 1.7566368e-03],
       [1.6795322e-02, 5.4620668e-02, 1.2594332e-02, ..., 3.3152729e-01,
        5.7644071e-03, 4.3009254e-03],
       ...,
       [4.7781598e-03, 2.3181826e-02, 3.7301569e-03, ..., 2.6793128e-01,
        1.5019544e-03, 8.6596690e-04],
       [1.2502580e-02, 4.4389904e-02, 9.3440134e-03, ..., 3.1676495e-01,
        4.1972105e-03, 2.9030817e-03],
       [2.2409065e-03, 1.3961519e-02, 1.7897844e-03, ..., 2.3365331e-01,
        6.7403406e-04, 3.3742326e-04]], dtype=float32)

In [ ]:
model_1.evaluate(padded_docs_test_1,y_test)

70/70 [==============================] - 3s 40ms/step - loss: 0.1401 - binary_accuracy: 0.9594


[0.1400931179523468, 0.9594241380691528]

In [ ]:
t = train.sample(1)
encoded_docs_1 = vect.texts_to_sequences(t['preprocessed_title'])
padded_docs_1 = pad_sequences(encoded_docs_1, maxlen=1200)
pred = model_1.predict(padded_docs_1).tolist()
for i in range(len(pred[0])):
  if(pred[0][i] < 0.1):
    pred[0][i] =0
  else:
    pred[0][i] =1
print("Original tags -->", t['tags'].values)
print("Predicted tags -->", vectorizer.inverse_transform(pred[0])[0])

Original tags --> ['plottwist,revenge,cruelty,violence,romantic']
Predicted tags --> ['cult' 'flashback' 'murder' 'psychedelic' 'revenge' 'romantic' 'violence']


In [ ]:
pred=predictions_1.copy()
pred[pred>=0.1]=1
pred[pred<0.1]=0
pred=pred.astype(np.int64)
from sklearn.metrics import hamming_loss
hamming_loss(y_test,pred)

0.10687767051748695

In [ ]:
df["total"]=df["preprocessed_title"]+df["preprocessed_plots"]

In [ ]:
df["total"]

0        tre volti della pauranote synopsis orginal ita...
1        dungeons dragons book vile darknesstwo thousan...
2        shop around cornermatuschek gift store budapes...
3        mr holland opusglenn holland morning person an...
4        scarfacemay cuban man named tony montana al pa...
                               ...                        
14823    lucky numbersruss richards john travolta weath...
14824    iron manrussia media covers tony stark disclos...
14825    play dirtynorth african campaign world war ii ...
14826    high wallsteven kenet catches unfaithful wife ...
14827    hopesometime chicago man cecil moe michael mad...
Name: total, Length: 14828, dtype: object

In [ ]:
train, validate, test = np.split(df.sample(frac=1), [int(.7*len(df)), int(.85*len(df))])

In [ ]:
train

,imdb_id,title,plot_synopsis,tags,split,synopsis_source,preprocessed_plots,preprocessed_title,total
6950,tt0028333,Swing Time,"John ""Lucky"" Garnett (Fred Astaire) is a gambl...",romantic,test,wikipedia,john lucky garnett fred astaire gambler dancer...,swing time,swing timejohn lucky garnett fred astaire gamb...
7706,tt0044953,The Naked Spur,"In March 1868, Howard Kemp (James Stewart) is ...","goodversusevil,violence,murder",train,wikipedia,march howard kemp james stewart tracking ben v...,naked spur,naked spurmarch howard kemp james stewart trac...
8997,tt1468829,Cherry Tree Lane,"In a house at Cherry Tree Lane, distant couple...","revenge,sadist",train,wikipedia,house cherry tree lane distant couple christin...,cherry tree lane,cherry tree lanehouse cherry tree lane distant...
14346,tt0234853,The Tao of Steve,Dex (Logue) is an unlikely Lothario - an overw...,"romantic,philosophical",train,wikipedia,dex logue unlikely lothario overweight thirtys...,tao steve,tao stevedex logue unlikely lothario overweigh...
3941,tt1427298,The Human Race,"An exciting, unpredictable and emotionally dra...",violence,test,imdb,exciting unpredictable emotionally dramatic ho...,human race,human raceexciting unpredictable emotionally d...
...,...,...,...,...,...,...,...,...,...
5560,tt0065466,Beyond the Valley of the Dolls,"The film concerns three young women, Kelly McN...","pornographic,murder,cult,violence,absurd,humor...",train,imdb,film concerns three young women kelly mcnamara...,beyond valley dolls,beyond valley dollsfilm concerns three young w...
7056,tt0159401,The Debt Collector,The film opens in late 1970s Edinburgh; Nicky ...,"revenge,murder,violence",train,wikipedia,film opens late edinburgh nicky dryden billy c...,debt collector,debt collectorfilm opens late edinburgh nicky ...
1921,tt0095294,Hellbound: Hellraiser II,The movie opens with Captain Elliott Spencer (...,"gothic,murder,cult,violence,horror,goodversuse...",test,imdb,movie opens captain elliott spencer doug bradl...,hellbound hellraiser ii,hellbound hellraiser iimovie opens captain ell...
14681,tt0118949,Into Thin Air: Death on Everest,The film portrays the 1996 Mount Everest disas...,tragedy,train,wikipedia,film portrays mount everest disaster events pr...,thin air death everest,thin air death everestfilm portrays mount ever...


In [ ]:
train=train.reset_index()
test=test.reset_index()
validate=validate.reset_index()

In [ ]:
train

,index,imdb_id,title,plot_synopsis,tags,split,synopsis_source,preprocessed_plots,preprocessed_title,total
0,6950,tt0028333,Swing Time,"John ""Lucky"" Garnett (Fred Astaire) is a gambl...",romantic,test,wikipedia,john lucky garnett fred astaire gambler dancer...,swing time,swing timejohn lucky garnett fred astaire gamb...
1,7706,tt0044953,The Naked Spur,"In March 1868, Howard Kemp (James Stewart) is ...","goodversusevil,violence,murder",train,wikipedia,march howard kemp james stewart tracking ben v...,naked spur,naked spurmarch howard kemp james stewart trac...
2,8997,tt1468829,Cherry Tree Lane,"In a house at Cherry Tree Lane, distant couple...","revenge,sadist",train,wikipedia,house cherry tree lane distant couple christin...,cherry tree lane,cherry tree lanehouse cherry tree lane distant...
3,14346,tt0234853,The Tao of Steve,Dex (Logue) is an unlikely Lothario - an overw...,"romantic,philosophical",train,wikipedia,dex logue unlikely lothario overweight thirtys...,tao steve,tao stevedex logue unlikely lothario overweigh...
4,3941,tt1427298,The Human Race,"An exciting, unpredictable and emotionally dra...",violence,test,imdb,exciting unpredictable emotionally dramatic ho...,human race,human raceexciting unpredictable emotionally d...
...,...,...,...,...,...,...,...,...,...,...
10374,5560,tt0065466,Beyond the Valley of the Dolls,"The film concerns three young women, Kelly McN...","pornographic,murder,cult,violence,absurd,humor...",train,imdb,film concerns three young women kelly mcnamara...,beyond valley dolls,beyond valley dollsfilm concerns three young w...
10375,7056,tt0159401,The Debt Collector,The film opens in late 1970s Edinburgh; Nicky ...,"revenge,murder,violence",train,wikipedia,film opens late edinburgh nicky dryden billy c...,debt collector,debt collectorfilm opens late edinburgh nicky ...
10376,1921,tt0095294,Hellbound: Hellraiser II,The movie opens with Captain Elliott Spencer (...,"gothic,murder,cult,violence,horror,goodversuse...",test,imdb,movie opens captain elliott spencer doug bradl...,hellbound hellraiser ii,hellbound hellraiser iimovie opens captain ell...
10377,14681,tt0118949,Into Thin Air: Death on Everest,The film portrays the 1996 Mount Everest disas...,tragedy,train,wikipedia,film portrays mount everest disaster events pr...,thin air death everest,thin air death everestfilm portrays mount ever...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(","), binary='true')
y_train = vectorizer.fit_transform(train['tags']).toarray()
y_test = vectorizer.transform(test['tags']).toarray()
y_val = vectorizer.transform(validate['tags']).toarray()

In [ ]:
vect=Tokenizer()
vect.fit_on_texts(train['total'])
vocab_size = len(vect.word_index) + 1

In [ ]:
encoded_docs_train = vect.texts_to_sequences(train['total'])
max_length = vocab_size
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=1200)
print(padded_docs_train)

[[    0     0     0 ...   143   193  5162]
 [    0     0     0 ...   120 17488   109]
 [    0     0     0 ...  2966  3747    27]
 ...
 [    0     0     0 ...    33    68   157]
 [    0     0     0 ... 11037    88   353]
 [    0     0     0 ...  2243   438   965]]


In [ ]:
encoded_docs_test =  vect.texts_to_sequences(test['total'])
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=1200)
encoded_docs_cv = vect.texts_to_sequences(validate['total'])
padded_docs_cv = pad_sequences(encoded_docs_cv, maxlen=1200)

In [ ]:
model = keras.Sequential()
model.add(layers.Embedding(vocab_size, output_dim=50, input_length=1200))
model.add(layers.LSTM(128, return_sequences=True))  
model.add(layers.Dropout(0.5))
model.add(layers.LSTM(64))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(71, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1200, 50)          5702800   
_________________________________________________________________
lstm_4 (LSTM)                (None, 1200, 128)         91648     
_________________________________________________________________
dropout_4 (Dropout)          (None, 1200, 128)         0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 71)                4615      
Total params: 5,848,471
Trainable params: 5,848,471
Non-trainable params: 0
____________________________________________

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['binary_accuracy'])

In [ ]:
history = model.fit(padded_docs_train,y_train,
                        epochs = 5,
                        verbose = 1,
                        validation_data=(padded_docs_test, y_test),
                        batch_size=16)

Epoch 1/5
649/649 [==============================] - 86s 129ms/step - loss: 0.2339 - binary_accuracy: 0.9214 - val_loss: 0.1359 - val_binary_accuracy: 0.9595
Epoch 2/5
649/649 [==============================] - 82s 126ms/step - loss: 0.1480 - binary_accuracy: 0.9570 - val_loss: 0.1358 - val_binary_accuracy: 0.9595
Epoch 3/5
649/649 [==============================] - 82s 127ms/step - loss: 0.1451 - binary_accuracy: 0.9569 - val_loss: 0.1359 - val_binary_accuracy: 0.9595
Epoch 4/5
649/649 [==============================] - 82s 126ms/step - loss: 0.1384 - binary_accuracy: 0.9577 - val_loss: 0.1376 - val_binary_accuracy: 0.9595
Epoch 5/5
649/649 [==============================] - 82s 126ms/step - loss: 0.1324 - binary_accuracy: 0.9585 - val_loss: 0.1395 - val_binary_accuracy: 0.9595


In [ ]:
predictions=model.predict(padded_docs_test)

In [ ]:
predictions

array([[0.00870052, 0.02943845, 0.00438499, ..., 0.23602542, 0.00274666,
        0.00170654],
       [0.01794238, 0.04855238, 0.00825829, ..., 0.28599817, 0.00574382,
        0.00400135],
       [0.01096253, 0.03452777, 0.00538247, ..., 0.2555251 , 0.00341646,
        0.00225396],
       ...,
       [0.00424651, 0.01777786, 0.00233261, ..., 0.19925238, 0.00122077,
        0.00070279],
       [0.00940872, 0.03108484, 0.00460059, ..., 0.24269396, 0.00292687,
        0.00184894],
       [0.0337458 , 0.07637337, 0.01549276, ..., 0.34739453, 0.01039278,
        0.00873936]], dtype=float32)

In [ ]:
model.evaluate(padded_docs_test,y_test)

70/70 [==============================] - 3s 40ms/step - loss: 0.1395 - binary_accuracy: 0.9595


[0.13951604068279266, 0.959474503993988]

In [ ]:
t = train.sample(1)
encoded_docs = vect.texts_to_sequences(t['total'])
padded_docs = pad_sequences(encoded_docs, maxlen=1200)
pred = model.predict(padded_docs).tolist()
for i in range(len(pred[0])):
  if(pred[0][i] < 0.1):
    pred[0][i] =0
  else:
    pred[0][i] =1
print("Original tags -->", t['tags'].values)
print("Predicted tags -->", vectorizer.inverse_transform(pred[0])[0])

Original tags --> ['murder']
Predicted tags --> ['cult' 'flashback' 'murder' 'revenge' 'romantic' 'violence']


In [ ]:
pred=predictions.copy()
pred[pred>=0.1]=1
pred[pred<0.1]=0
pred=pred.astype(np.int64)
from sklearn.metrics import hamming_loss
hamming_loss(y_test,pred)

0.10385187529672417

In [ ]:
pip install requests

In [ ]:
titles=[]
summary=[]
genres=[]
import requests
from bs4 import BeautifulSoup
from sklearn import tree
import json
headers = {'Accept-Language': 'en-US, en;q=0.5'}
page = requests.get("https://www.imdb.com/chart/top?sort=us,desc&mode=simple&page=1",headers=headers)
soup = BeautifulSoup(page.text, 'html.parser')
movies = soup.find_all("td", class_="titleColumn")
ctr=0
for movie in movies:
  movie_title = movie.a.contents[0]
  movie_url = 'http://www.imdb.com' + movie.a['href']
  titles.append(movie_title)
  #print(movie_title)
  movie_page = requests.get(movie_url) 
  soup = BeautifulSoup(movie_page.text, 'html.parser')
  g=json.loads(soup.find_all('script')[11].contents[0])["genre"]
  #print(g)
  if(type(g)==list):
    genres.append(','.join(g))
  else:
    genres.append(g)
  movie_summary=soup.find("div", class_="inline canwrap").find('span').text.strip()
  summary.append(movie_summary)
  #print(movie_summary)
  ctr=ctr+1
  if (ctr==250):
    break;   

In [ ]:
scrape_data=pd.DataFrame({'movie':titles,'plot':summary,'genres':genres})

In [ ]:
scrape_data.head

<bound method NDFrame.head of                             movie  ...                                           genres
0    Zack Snyder's Justice League  ...                  Action,Adventure,Fantasy,Sci-Fi
1                            Soul  ...  Animation,Adventure,Comedy,Family,Fantasy,Music
2                        Hamilton  ...                  Biography,Drama,History,Musical
3                      The Father  ...                                            Drama
4                            1917  ...                               Drama,Thriller,War
..                            ...  ...                                              ...
245                    Metropolis  ...                                     Drama,Sci-Fi
246                   The General  ...                Action,Adventure,Comedy,Drama,War
247                 The Gold Rush  ...                   Adventure,Comedy,Drama,Western
248                  Sherlock Jr.  ...                            Action,Comedy,Romance
24

In [ ]:
import re
import contractions
from bs4 import BeautifulSoup
preprocessed=[]
for i in scrape_data['plot'].values:
    i= re.sub(r"http\S+","", i)
    i= BeautifulSoup(i, 'lxml').get_text()
    i= re.sub("\S*\d\S*", "",i).strip()
    i= re.sub('[^A-Za-z]+', ' ', i)
    i= contractions.fix(i)
    i= ' '.join(j.lower() for j in i.split() if j.lower() not in stopwords)
    preprocessed.append(i.strip())
scrape_data['preprocessed_plots']=preprocessed

In [ ]:
check=["one","two","three","four","five","six","seven","eight","nine","ten","iii","i","ii","iv","v","vi","vii","viii","ix","x"]
preprocessed_title=[]
for i in scrape_data['movie'].values:
    i= re.sub(r"http\S+","", i)
    i= BeautifulSoup(i, 'lxml').get_text()
    i= re.sub("\S*\d\S*", "",i).strip()
    i= re.sub('[^A-Za-z]+', ' ', i)
    i= contractions.fix(i)
    i= ' '.join(j.lower() for j in i.split() if j.lower() not in stopwords)
    g=i.split()
    if("part" in g):
      ind=g.index("part")
      if((ind+1<len(g)-1) and (g[ind+1] in check)):
        g.remove(g[ind])
        g.remove(g[ind+1])
    i= ' '.join(g)
    preprocessed_title.append(i.strip())
scrape_data['preprocessed_title']=preprocessed_title

In [ ]:
def remove_spaces(x):
    return x.replace(" ", "")
scrape_data['genres']=scrape_data['genres'].apply(remove_spaces)
scrape_data['genres']

0                      Action,Adventure,Fantasy,Sci-Fi
1      Animation,Adventure,Comedy,Family,Fantasy,Music
2                      Biography,Drama,History,Musical
3                                                Drama
4                                   Drama,Thriller,War
                            ...                       
245                                       Drama,Sci-Fi
246                  Action,Adventure,Comedy,Drama,War
247                     Adventure,Comedy,Drama,Western
248                              Action,Comedy,Romance
249                                Comedy,Drama,Family
Name: genres, Length: 250, dtype: object

In [ ]:
g_init=set()
for i in df["tags"]:
  l=i.split(',')
  for j in l:
    g_init.add(j.lower())
g_new=set()
for i in scrape_data["genres"]:
  l=i.split(',')
  for j in l:
    g_new.add(j.lower())
g_2=g_init.intersection(g_new)
print(g_init)
print(g_new)
print(g_2)

{'intrigue', 'romantic', 'prank', 'plottwist', 'comedy', 'nonfiction', 'alternatehistory', 'magicalrealism', 'antiwar', 'atmospheric', 'adultcomedy', 'humor', 'pornographic', 'historicalfiction', 'allegory', 'whimsical', 'violence', 'depressing', 'dramatic', 'sentimental', 'satire', 'tragedy', 'goodversusevil', 'horror', 'bleak', 'homemovie', 'avantgarde', 'suicidal', 'western', 'action', 'clever', 'blaxploitation', 'haunting', 'comic', 'inspiring', 'gothic', 'christianfilm', 'entertaining', 'neonoir', 'absurd', 'grindhousefilm', 'philosophical', 'revenge', 'psychedelic', 'boring', 'storytelling', 'cute', 'insanity', 'claustrophobic', 'realism', 'brainwashing', 'sadist', 'queer', 'paranormal', 'alternatereality', 'cruelty', 'melodrama', 'historical', 'flashback', 'autobiographical', 'feel-good', 'suspenseful', 'mystery', 'dark', 'psychological', 'murder', 'stupid', 'sci-fi', 'fantasy', 'thought-provoking', 'cult'}
{'comedy', 'history', 'biography', 'crime', 'animation', 'musical', 'thr

In [ ]:
for index,row in scrape_data["genres"].iteritems():
  x=row.split(",")
  for j in x:
    if(j not in g_2):
      x.remove(j)
  scrape_data.at[index,'genres']=",".join(x)
scrape_data

,movie,plot,genres,preprocessed_plots,preprocessed_title
0,Zack Snyder's Justice League,Determined to ensure Superman's ultimate sacri...,"Adventure,Sci-Fi",determined ensure superman ultimate sacrifice ...,zack snyder justice league
1,Soul,Joe is a middle-school band teacher whose life...,"Adventure,Family,Music",joe middle school band teacher whose life quit...,soul
2,Hamilton,"""Hamilton"" is the story of America then, told ...","Drama,Musical",hamilton story america told america featuring ...,hamilton
3,The Father,A man refuses all assistance from his daughter...,,man refuses assistance daughter ages tries mak...,father
4,1917,"April 1917, the Western Front. Two British sol...",Thriller,april western front two british soldiers sent ...,
...,...,...,...,...,...
245,Metropolis,"Sometime in the future, the city of Metropolis...",Sci-Fi,sometime future city metropolis home utopian s...,metropolis
246,The General,"Johnnie loves his train (""The General"") and An...","Adventure,Drama",johnnie loves train general annabelle lee civi...,general
247,The Gold Rush,A lone prospector ventures into Alaska looking...,"Comedy,Western",lone prospector ventures alaska looking gold g...,gold rush
248,Sherlock Jr.,"A meek and mild projectionist, who also cleans...",Comedy,meek mild projectionist also cleans screenings...,sherlock jr


In [ ]:
scrape_data["total"]=scrape_data["preprocessed_title"]+scrape_data["preprocessed_plots"]

In [ ]:
y_test= vectorizer.transform(scrape_data['genres']).toarray()

In [ ]:
vect=Tokenizer()
vect.fit_on_texts(scrape_data['total'])
vocab_size = len(vect.word_index) + 1
vocab_size

5961

In [ ]:
encoded_docs_test =  vect.texts_to_sequences(scrape_data['total'])
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=1200)

In [ ]:
predictions=model.predict(padded_docs_test)

In [ ]:
model.evaluate(padded_docs_test,y_test)

8/8 [==============================] - 0s 39ms/step - loss: 0.0507 - binary_accuracy: 0.9964


[0.05066792294383049, 0.9963943958282471]